Importandos todas as Bibliotecas necessárias.

In [2]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx2pdf import convert

Classe que realiza a busca de dados na planilha do Excel e retorna uma Series do Pandas:<br>
 * Se não for especificada a aba, ele irá buscar em todas as abas até encontrar a primeira ocorrência;
 
 * Importante analisar a ordem das abas, abas com valores únicos devem ser as primeiras da planilha;

 * Se na célular houver valores separados por ';' será criada uma lista com os valores separados pelo ';'.

In [73]:
class BuscarDados:

    def __init__(self, valor, aba=None):
        
        if not aba:
            banco_de_dados = pd.read_excel('01_Cadastro.xlsm', sheet_name=None)
            for dados_aba in banco_de_dados.values():
                if valor in dados_aba.values:
                    banco_de_dados = dados_aba
                    break
            else:
                raise ValueError('O valor não foi encontrado nas abas do arquivo')

        else:
            banco_de_dados = pd.read_excel('01_Cadastro.xlsm', sheet_name = aba)
            if valor not in banco_de_dados.values:
                raise ValueError('O valor não foi encontrado nas abas do arquivo')
        
        linha = banco_de_dados.loc[banco_de_dados.isin([valor]).any(axis = 1)].iloc[0].astype(str)
        self.dicionario = {indice: valor.split(';') if ';' in valor
                            else valor for indice, valor in linha.items()}

In [74]:
BuscarDados(2,'Peticoes').dicionario


c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


{'ID_PETICAO': '2',
 'PROCESSO': 'inicial',
 'CLIENTE': '0',
 'ADVERSO': '0',
 'PECA': 'GFA_Execucao-de-condominio',
 'ATRIBUTOS': ['Unidade_Devedora', 'Matricula_do_Imovel', 'CRI', 'Debito'],
 'VALORES': ['12', '12', 'abc', 'zzz']}

Classe que cria um dicionária da Parte procurada:
* As chaves do dicionário são os atributos da parte, com seus valores como os valores;

* Qualquer atributo pode ser usado para buscar a parte (nome, cpf, cnpj, etc...).

In [61]:
class QualificarParte:
    
    def __init__(self, parte):
        
        self.dicionario = BuscarDados(parte).dicionario
        self._agrupar_endereco()
        self._qualificar_parte()
          
    def _agrupar_endereco(self):

        self.dicionario['NUMERO_COMPLEMENTO'] = (f"{self.dicionario['NUMERO']} - {self.dicionario['COMPLEMENTO']}"
                                                if self.dicionario['COMPLEMENTO'] is True
                                                else self.dicionario['NUMERO'])
                
        self.dicionario['ENDERECO'] = '{}, nº {}, Bairro {}, CEP {}, {}/{}'.format(
                                        self.dicionario['LOGRADOURO'], self.dicionario['NUMERO_COMPLEMENTO'],
                                        self.dicionario['BAIRRO'], self.dicionario['CEP'],
                                        self.dicionario['CIDADE'], self.dicionario['UF']
                                        )
   
    def _qualificar_parte(self):

        if 'CPF' in self.dicionario:
            self.dicionario['QUALIFICAÇÃO'] = '{}, {}, {}, {}, RG n° {} {}, CPF n° {}, e-mail: {}, com endereço à {}'.format(
                                                self.dicionario['NOME'], self.dicionario['NACIONALIDADE'], self.dicionario['ESTADO_CIVIL'],
                                                self.dicionario['PROFISSAO'], self.dicionario['RG'], self.dicionario['EXPEDIDOR_RG'],
                                                self.dicionario['CPF'], self.dicionario['EMAIL'],self.dicionario['ENDERECO']
                                                )

        elif self.dicionario['TAG'] == 'condominio':
            self.dicionario['QUALIFICAÇÃO'] = ('{}, inscrito no CNPJ n° {}, com endereço à {}, devidamente representado neste ato '
                                               'pelo seu síndico(a) {}'.format(
                                                self.dicionario['NOME'], self.dicionario['CNPJ'],
                                                self.dicionario['ENDERECO'], self.dicionario['ADMINISTRADOR'])
                                                )
            
        else:
            self.dicionario['QUALIFICAÇÃO'] = ('{}, pessoa jurídica inscrita no CNPJ n° {}, com endereço à {}, devidamente '
                                                'representada neste ato pelo(a) sócio(a) {}').format(
                                                self.dicionario['NOME'], self.dicionario['CNPJ'],
                                                self.dicionario['ENDERECO'], self.dicionario['ADMINISTRADOR']
                                                )

Classe que cria um dicionário do documento procurado:
* O documento deve ser procurado pelo ID (atributo único dele);

* O cliente ou o adverso podem ser mais de uma pessoa.

In [98]:
x = CriarDocumento(1)
x.dicionario

c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


{'ID_PETICAO': '1',
 'PROCESSO': 'inicial',
 'CLIENTE': 'Edificio Vivare Residence',
 'ADVERSO': '0',
 'PECA': 'GFA_Procuracao-de-condominio',
 'U': '1',
 'N': '1'}

In [95]:
class CriarDocumento:
            
    def __init__(self, documento):
        
        self.dicionario = BuscarDados(documento, 'Peticoes').dicionario
        self.dicionario.update(dict(zip(self.dicionario.pop('ATRIBUTOS'), self.dicionario.pop('VALORES'))))

    '''

    for chave, valor in QualificarParte(self.dicionario['CLIENTE']).dicionario.items():
            nova_chave = 'CLIENTE_' + chave
            self.dicionario[nova_chave] = valor

        if self.dicionario['ADVERSO'] == True:
            for chave, valor in QualificarParte(self.dicionario['ADVERSO']).dicionario.items():
                nova_chave = 'ADVERSO_' + chave
                self.dicionario[nova_chave] = valor


        if self.dicionario['ADVERSO'] == True:
            self._atualizar_dicionario_com_parte(self.dicionario['ADVERSO'], 'ADVERSO_')  
    
    def _atualizar_dicionario_com_parte(self, parte_do_documento, prefixo = 'CLIENTE_'):

        def dicionario_se_parte_unica(parte_do_documento, prefixo):
            for chave, valor in QualificarParte(parte_do_documento).dicionario.items():
                nova_chave = prefixo + chave
                self.dicionario[nova_chave] = valor
        
        #checa o número de parte se é multiplo(lista)#       
        if isinstance(parte_do_documento, str):            
            dicionario_se_parte_unica(parte_do_documento, prefixo)
            
        else:
            dicionario_se_parte_unica(parte_do_documento[0], prefixo)
            self.dicionario['CLIENTE_QUALIFICAÇÃO'] = f"(1) {self.dicionario['CLIENTE_QUALIFICAÇÃO']}"

            for index, nome in enumerate(parte_do_documento[1:], start=2):
                self.dicionario['CLIENTE_QUALIFICAÇÃO'] += f" e ({index}) {QualificarParte(nome).dicionario['QUALIFICAÇÃO']}"    '''

In [90]:
x = CriarDocumento(2)
x.dicionario


c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


{'ID_PETICAO': '2',
 'PROCESSO': 'inicial',
 'CLIENTE': '0',
 'ADVERSO': '0',
 'PECA': 'GFA_Execucao-de-condominio',
 'ATRIBUTOS': ['Unidade_Devedora', 'Matricula_do_Imovel', 'CRI', 'Debito'],
 'VALORES': ['12', '12', 'abc', 'zzz'],
 'Unidade_Devedora': '12',
 'Matricula_do_Imovel': '12',
 'CRI': 'abc',
 'Debito': 'zzz'}

In [381]:
class iniciar_documento:
            
    def __init__(self, documento, cliente, adverso = None):        
        
        self.dicionario = BuscarDados(documento, 'Peticoes').dicionario
        self._carregar_atributos_e_valores(documento)
        self._criar_dicionario(cliente, adverso)
       

    def _carregar_atributos_e_valores(self, documento):
        self.atributos = self.dicionario['ATRIBUTOS'].strip().upper().split('/')
        self.valores = self.dicionario['VALORES'].strip().split('/')
       
    def _criar_dicionario(self, cliente, adverso):
        self.dicionario = dict(zip(self.atributos, self.valores))
        self._atualizar_dicionario_com_parte(cliente)
        if adverso:
            self._atualizar_dicionario_com_parte(adverso, 'ADVERSO_')  
    
    def _atualizar_dicionario_com_parte(self, parte_do_documento, prefixo = 'CLIENTE_'):

        def dicionario_se_parte_unica(parte_do_documento, prefixo):
            for chave, valor in QualificarParte(parte_do_documento).dicionario.items():
                nova_chave = prefixo + chave
                self.dicionario[nova_chave] = valor
        
        #checa o número de parte se é multiplo(lista)#       
        if isinstance(parte_do_documento, str):            
            dicionario_se_parte_unica(parte_do_documento, prefixo)
            
        else:
            dicionario_se_parte_unica(parte_do_documento[0], prefixo)
            self.dicionario['CLIENTE_QUALIFICAÇÃO'] = f"(1) {self.dicionario['CLIENTE_QUALIFICAÇÃO']}"

            for index, nome in enumerate(parte_do_documento[1:], start=2):
                self.dicionario['CLIENTE_QUALIFICAÇÃO'] += f" e ({index}) {QualificarParte(nome).dicionario['QUALIFICAÇÃO']}"    

In [309]:
criar_docx_e_pdf('GFA_Procuracao-de-condominio', 'Residencial Luiz Roberto Luize')

c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


AttributeError: 'float' object has no attribute 'strip'

In [308]:
#CORRIGR A BUSCA DO DOCUMENTO, TEM QUE SER PELO NUMERO DA PETICAO

class criar_docx_e_pdf:

    def __init__(self, nome_documento, cliente, adverso = None):

        dicionario = iniciar_documento(nome_documento, cliente, adverso).dicionario
        
        docx = Document(f"01_Modelos\{nome_documento}.docx")
         
        chave_de_substituicao = {'x' + atributo + 'x': str(valor) for atributo, valor in dicionario.items()}
               
        for paragrafo in docx.paragraphs:
            for chave in chave_de_substituicao:
                if chave in paragrafo.text:
                    paragrafo.text = paragrafo.text.replace(chave, chave_de_substituicao[chave])
                      
        try:
            nome_salvo = f"02_Documentos-prontos\{nome_documento} - {dicionario['CLIENTE_NOME']} x {dicionario['ADVERSO_NOME']}"
            #docx.save(f"{nome_salvo}.docx")
            #convert(f"02_Documentos-prontos\{nome_salvo}.docx", f"02_Documentos-prontos\{nome_salvo}.pdf")
        except:
            nome_salvo = f"02_Documentos-prontos\{nome_documento} - {dicionario['CLIENTE_NOME']}"
        docx.save(f"{nome_salvo}.docx")
        convert(f"02_Documentos-prontos\{nome_salvo}.docx", f"02_Documentos-prontos\{nome_salvo}.pdf")

        #Convertendo arquivo word em pdf        
        #time.sleep(0.93)
        #convert(f"02_Documentos-prontos\{nome_salvo}.docx", f"02_Documentos-prontos\{nome_salvo}.pdf")

In [10]:
class processo:

    def __init__(self, processo):

        self.dicionario = busca_dados(processo).linha.to_dict()

class peca:

    def __init__(self, peca):

        self.dicionario = busca_dados(peca).linha.to_dict()

In [45]:
#TESTE#
lista = ['Residencial Luiz Roberto Luize', 'Edificio Vivare Residence', 'Condominio Edificio Melissa', 'Edifício Anthuriums Place']
cliente = 'Residencial Luiz Roberto Luize'

for i in lista:
    peca('GFA - Procuração de Condomínio', i)

#x = peca('GFA - Procuração de Condomínio', lista)
#x = PECA('Procuração', lista)

#print(type(lista))

c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Alberto\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\A

In [4]:
from num2words import num2words

numero = 23
extenso = num2words(numero, lang='pt_BR')

print(f'{extenso} reais e {extenso} centavos')


vinte e três reais e vinte e três centavos


In [11]:
import pyodbc

dados_conexao = ('Driver={SQL Server};Server=DESKTOP-SMFGJPJ\SQLEXPRESS;Database=teste')
conexao = pyodbc.connect(dados_conexao)
print('Conexão Bem sucedida')
cursor = conexao.cursor()
cursor.execute("SELECT * FROM nome")
cursor.commit()
cursor.close()
conexao.close()


Conexão Bem sucedida


In [ ]:
cursor.execute("INSERT INTO tabela_origem (campo1, campo2) VALUES (?, ?)", valor1, valor2)

# Obter o ID do último registro inserido
cursor.execute("SELECT SCOPE_IDENTITY() AS LastID")
id_origem = cursor.fetchone().LastID

# Inserir dados na tabela de destino usando o ID da tabela de origem
cursor.execute("INSERT INTO tabela_destino (id_origem, campo3, campo4) VALUES (?, ?, ?)", id_origem, valor3, valor4)

# Confirmar as alterações
cursor.commit()


